# Exploration of Published Models

(discussion)

## Structural model

The structural model reported in [1] is a system of ordindary differential and algebraic equations. It combines a mechanistic description of the pharmacokinetics (PK) and the pharacodynamics (PD) of the compound, as well as an empirical model of the tumour growth.

The distribution of the compound in the mouse system after an oral dose is modelled by a one-compartment PK model with indirect drug administration

\begin{align*}
\frac{\text{d}A_d}{\text{d}t} = r_d(t) - k_{a}A_d,\qquad \frac{\text{d}A_c}{\text{d}t} = k_{a} A_d - k_e A_c,\qquad C_c = \frac{A_c}{V_c}.
\end{align*}

Here, $A_d$ is the amount of the compound in the dose compartment, and $A_c$ the amount in the central compartment. The dose compartment may be interpreted as the gut compartment. As the drug is orally administered the uptake into the dose compartment may be modelled by a time dependent dose rate $r_d$. The drug diffuses at a constant rate $k_a$ (absorption rate) into the central compartment. The central compartment models the drug circulating in the plasma. The clearance of the drug through the liver is modelled with a linear elimination rate $k_e$ from the central compartment. The concentration of the compound in the central compartment $C_c$ can be derived by normalising the amount of the drug by its effective volume of distribution in the central compartment $V_c$.

The tumour growth is modelled by an empirical model in [1]. In absence of the drug the tumour growth is modelled by an exponential growth phase for small tumour volumes, and by a linear growth phase for large tumour volumes. The growth inhibiting effects of the compound are assumed to be proportional to its concentration in central compartment and the volume of the tumour

\begin{align*}
\frac{\text{d}V^s_T}{\text{d}t} = \frac{2\lambda _0\lambda _1 V^s_T}{2\lambda _0 V^s_T + \lambda _1} - \kappa C_cV^s_T.
\end{align*}

Here, $V^s_T$ is the predicted tumour volume by the structural model, $\lambda _0$ is the exponential growth rate, $\lambda _1$ is the linear growth rate, and $\kappa $ is the growth inhibiting potency of the compound.

In [1], the dose and central compartment were assumed to contain no amount of the compound initially, $A_d(t=0)=0$ and $A_c(t=0)=0$. As the dose rate $r_t$ is determined by the dosing regimen, this leaves the structural model with seven parameters that determine the PKPD of the drug

\begin{align*}
\psi = (k_a, k_e, V_c, V_0, \lambda _0, \lambda _1, \kappa ),
\end{align*}

where $V_0$ is the initial tumour volume, $V^s_T(t=0)=V_0$.

We have implemented the structural model in a separate python [module](https://github.com/DavAug/ErlotinibGefitinib/blob/master/pkpd/model.py), such that its myokit representation can be created with `pkpd.model.create_pktgi_model()`.

In [1]:
#
# Create structural model.
#

import pkpd.model as m


# Create model
model = m.create_pktgi_model()

# Set route of administration to 'oral dosing'
model.set_roa(dose_comp='central', indirect=True)

# Show model
print(model.code())

[[model]]
# Initial values
central.amount   = 0
central.volume_t = 0
depot.amount     = 0

[central]
dot(amount) = -k_e * amount + depot.k_a * depot.amount
    in [mg]
conc = amount / volume_c
    in [mg/L]
k_e = 0
    in [1/day]
kappa = 0
    in [L/mg/day]
lambda_0 = 0
    in [1/day]
lambda_1 = 1
    in [cm^3/day]
time = 0 bind time
    in [day]
volume_c = 1
    in [L]
dot(volume_t) = 2 * (lambda_0 * (lambda_1 * volume_t)) / (2 * (lambda_0 * volume_t) + lambda_1) - kappa * (conc * volume_t)
    in [cm^3]

[depot]
dot(amount) = -k_a * amount + dose_rate * regimen
    in [mg]
dose_rate = 0
    in [mg/day]
k_a = 0
    in [1/day]
regimen = 0 bind pace
    in [1]




## Error model

The expected random deviations of the observed tumour volumes from the structural model predictions $V^s_T$ are modelled in [1] by a combined error model. A combined error model assumes that the deviations $\varepsilon $ of the observable tumour volume $V_T$ from the structural model predictions $V^s_T$ may be modelled by a base-level Gaussian noise, and a Gaussian noise contribution whose standard deviation grows relative to the structural model predictions

\begin{equation*}
    \varepsilon = \left(\sigma _{\text{base}} + \sigma _{\text{rel}} V^s_T\right) \varepsilon _n.
\end{equation*}

Here, $\sigma _{\text{base}}$ is the standard deviation of the base-level noise, and $\sigma _{\text{rel}}$ is the standard deviation relative to $V^s_T$. $\varepsilon _n$ is a standard Gaussian random variable $\mathcal{N}(0, 1)$. As a result, the model predictions of the tumour volume are Gaussian-distributed, centered at the structural model predictions $V^s_T = V^s_T(t, r_d; \psi)$ and a standard deviation $\sigma _{\text{tot}} = \sigma _{\text{base}} + \sigma _{\text{rel}} V^s_T$

\begin{equation*}
    V_T \sim \mathcal{N}(V^s_T, \sigma ^2_{\text{tot}}).
\end{equation*}

The error model introduces two further parameters

\begin{equation*}
    \theta _V = (\sigma _{\text{base}}, \sigma _{\text{rel}}),
\end{equation*}

such that the PKPD model has nine parameters ($\psi$, $\theta _V$).

## Population model

In [1] it is assumed that some of the parameters $\psi $ can vary between individual mice, while the error model parameters $\theta _V$ remain the same. 

## Bibliography

- <a name="ref1"> [1] </a> Eigenmann et. al., Combining Nonclinical Experiments with Translational PKPD Modeling to Differentiate Erlotinib and Gefitinib, Mol Cancer Ther (2016)